# Import biblioteka

In [1]:
import pandas as pd
import matplotlib.pyplot as plt

from sklearn import metrics
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import  StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

import datetime

# Učitavanje podataka

In [2]:
X_test= pd.read_csv('X_test.txt', sep='\s+')
y_test= pd.read_csv('y_test.txt', sep='\s+')
X_train= pd.read_csv('X_train.txt', sep='\s+')
y_train= pd.read_csv('y_train.txt', sep='\s+')

In [3]:
podaci_test = pd.concat([X_test, y_test], axis=1)
podaci_train = pd.concat([X_train, y_train], axis=1)

In [4]:
podaci = pd.concat([podaci_train, podaci_test], axis=0)

In [5]:
def matrica_predv(conf_matrica):
    cm_df = pd.DataFrame(conf_matrica, index=[f"Стварна вриједност {i+1}" for i in range(6)], columns=[f"Предикција {i+1}" for i in range(6)])

    # Prikaz tabele
    return cm_df

In [6]:
def klas_izv (y_skup,X_skup_predikcija, skup):
    izvjestaj=metrics.classification_report(y_skup, X_skup_predikcija, digits=8,output_dict=True)
     
    izvjestaj_df=pd.DataFrame(izvjestaj).transpose()
    izvjestaj_df['support'] = izvjestaj_df['support'].round().astype(int)
    
    izvjestaj_df = izvjestaj_df.rename(columns={
            'precision': 'Прецизност',
            'recall': 'Одзив',
            'f1-score': 'f1-score',
            'support': 'Број узорака у класи'  # Ovaj naziv se koristi za broj uzoraka u svakoj klasi
        })

    izvjestaj_df = izvjestaj_df.rename(index={
            '1': 'Пјешачење',
            '2': 'Пењање уз степенице',
            '3': 'Силазак низ степенице',
            '4': 'Сједење',
            '5': 'Стајање',
            '6': 'Лежање',
            'accuracy': 'Тачност',
            'macro avg': 'Макро просјек',  #prosečne vrijednosti metrika za sve klase, bez ponderisanja u zavisnosti od broja uzoraka u svakoj klasi.
            'weighted avg': 'Пондерисани просјек'
        })
 
   
        
    return  izvjestaj_df

# Standardizacija 

Standardizacija podataka se radi da bi se obezbijedilo da pojedinačne vrijednosti podataka i jedinice u kojima su uizrađene ne utiču na učenje - nastoji se postići da podaci imaju normalnu raspodjelu - Gausovu raspodjelu sa srednjom vrijednošću 0 i varijansom 1.

U praksi transformišemo podatke kako bismo ih centrirali oduzimanjem srednje vrijednosti svake osobine, a zatim ih skaliramo dijeljenjem vrijednosti njihovom standardnom devijacijom.

In [7]:
#Standardizacija trening skupa

# Za kolonu Class ne radi se standardizacija

# Inicijalizacija StandardScaler
scaler = StandardScaler()

# Primjena standardizacije
st_X_train = scaler.fit_transform(X_train)

# Prevođenje podataka u dataframe
st_X_train = pd.DataFrame(st_X_train, columns=X_train.columns)

st_X_train

,tBodyAcc-mean()-X,tBodyAcc-mean()-Y,tBodyAcc-mean()-Z,tBodyAcc-std()-X,tBodyAcc-std()-Y,tBodyAcc-std()-Z,tBodyAcc-mad()-X,tBodyAcc-mad()-Y,tBodyAcc-mad()-Z,tBodyAcc-max()-X,...,fBodyBodyGyroJerkMag-meanFreq(),fBodyBodyGyroJerkMag-skewness(),fBodyBodyGyroJerkMag-kurtosis(),"angle(tBodyAccMean,gravity)","angle(tBodyAccJerkMean),gravityMean)","angle(tBodyGyroMean,gravityMean)","angle(tBodyGyroJerkMean,gravityMean)","angle(X,gravityMean)","angle(Y,gravityMean)","angle(Z,gravityMean)"
0,0.200642,-0.063683,-0.419628,-0.868814,-0.939441,-0.737529,-0.859817,-0.939019,-0.766437,-0.856036,...,-0.795359,0.025960,-0.276399,-0.360603,0.062940,-0.778427,-0.026080,-0.687219,0.407946,-0.007568
1,0.055948,0.031486,-0.253908,-0.875426,-0.923902,-0.849304,-0.868531,-0.921998,-0.848928,-0.871359,...,0.130614,-0.897357,-0.767990,0.133011,-0.021461,-1.218805,1.484470,-0.694138,0.409117,0.007875
2,0.073515,-0.043416,-0.076295,-0.869039,-0.907760,-0.893785,-0.863137,-0.898854,-0.896701,-0.863323,...,1.152336,-0.260878,-0.438316,-0.377840,0.391976,0.151207,1.704201,-0.702239,0.410288,0.026502
3,0.066696,-0.208422,-0.249712,-0.870626,-0.940022,-0.921805,-0.864503,-0.938124,-0.925279,-0.863323,...,1.112769,0.591045,0.463155,-0.135025,-0.033637,1.037851,-1.003019,-0.701684,0.414650,0.031714
4,0.030469,0.027587,-0.109848,-0.875188,-0.934878,-0.921343,-0.867384,-0.931789,-0.928028,-0.870260,...,-0.149577,-0.138515,-0.240313,0.340406,0.268486,1.125918,-1.276282,-0.700152,0.425463,0.045225
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7347,0.358361,-0.967904,-1.273005,0.913859,1.095963,1.628612,0.821169,1.174137,1.609686,1.247727,...,-0.778758,-0.876738,-0.829197,-0.591277,1.846034,0.325923,-0.878008,-0.590763,0.605162,0.380984
7348,-0.009044,0.243731,-0.676787,0.824887,1.026150,1.586100,0.726245,1.023755,1.658407,1.076279,...,0.159241,-0.260847,-0.180290,0.166951,1.948561,-1.459501,0.849894,-0.551598,0.652468,0.381820
7349,-0.015668,0.016781,1.132221,0.862975,0.810002,2.100249,0.768781,0.880813,2.266963,0.941403,...,0.277877,1.034784,1.044548,0.131018,-0.599877,1.406760,1.479571,-0.565848,0.640597,0.348709
7350,0.215866,-0.028123,-0.867710,0.860922,0.794902,2.086778,0.754697,0.944019,2.235301,1.047375,...,-0.443978,1.155541,0.913569,-0.326769,1.558312,1.525574,-1.220854,-0.577668,0.631478,0.293276


In [8]:
#Standardizacija testnog skupa

# Za kolonu Class ne radi se standardizacija

# Inicijalizacija StandardScaler
scaler = StandardScaler()

# Primjena standardizacije
st_X_test = scaler.fit_transform(X_test)

# Prevođenje podataka u dataframe
st_X_test = pd.DataFrame(st_X_test, columns=X_test.columns)


st_X_test

,tBodyAcc-mean()-X,tBodyAcc-mean()-Y,tBodyAcc-mean()-Z,tBodyAcc-std()-X,tBodyAcc-std()-Y,tBodyAcc-std()-Z,tBodyAcc-mad()-X,tBodyAcc-mad()-Y,tBodyAcc-mad()-Z,tBodyAcc-max()-X,...,fBodyBodyGyroJerkMag-meanFreq(),fBodyBodyGyroJerkMag-skewness(),fBodyBodyGyroJerkMag-kurtosis(),"angle(tBodyAccMean,gravity)","angle(tBodyAccJerkMean),gravityMean)","angle(tBodyGyroMean,gravityMean)","angle(tBodyGyroJerkMean,gravityMean)","angle(X,gravityMean)","angle(Y,gravityMean)","angle(Z,gravityMean)"
0,-0.277708,-0.210631,2.193076,-0.787267,-0.833212,-0.093444,-0.808091,-0.839013,-0.104178,-0.824746,...,-0.253663,-0.166391,-0.344765,0.003566,0.357573,-1.363900,0.575487,-0.404790,0.622723,-0.038348
1,0.198660,0.182591,-0.250264,-0.876984,-0.929061,-0.858050,-0.897145,-0.928949,-0.863269,-0.824746,...,-2.300750,0.491021,0.012257,-0.264092,0.030787,-0.747232,1.871192,-0.361739,0.636731,-0.145709
2,0.024587,-0.318055,-0.228485,-0.921598,-0.934054,-0.907107,-0.916891,-0.933813,-0.912639,-0.910982,...,-0.291563,0.274821,-0.134989,-0.119671,0.446072,0.037919,0.323938,-0.370932,0.632845,-0.126855
3,-0.061057,-0.573037,-0.213709,-0.923837,-0.940817,-0.919082,-0.919152,-0.941599,-0.927776,-0.909740,...,-0.058625,-0.210678,-0.441714,-0.066443,0.338512,0.472695,0.625874,-0.363434,0.645276,-0.117585
4,0.013823,-0.387881,-0.494639,-0.921679,-0.929036,-0.949979,-0.916133,-0.923846,-0.951396,-0.909740,...,-1.090781,-0.810527,-0.796939,-0.022276,-0.098529,1.097348,-0.201999,-0.350256,0.665657,-0.104118
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2942,0.597072,-1.380212,0.217067,0.789690,0.744135,1.154623,0.740507,0.779257,1.132204,1.235951,...,-0.241427,-0.311121,-0.488935,-1.019623,0.769650,1.330760,-1.359840,-0.270682,0.616018,0.967186
2943,1.476032,-0.829444,0.057817,0.747216,1.085580,1.206230,0.695181,1.026724,1.025466,1.235951,...,-0.122855,-0.135013,-0.326438,-2.207635,-0.846487,-1.098551,0.678029,-0.277456,0.612784,0.957360
2944,1.254458,1.862441,-0.173180,0.688432,0.943345,1.057851,0.657591,0.811598,0.810778,0.896354,...,-0.850662,0.500453,0.423089,-0.555875,0.190504,1.034263,0.759011,-0.277802,0.615561,0.952271
2945,-0.601088,1.411385,0.278129,0.704246,0.563664,1.175343,0.646262,0.506579,0.971675,0.896354,...,-0.765017,0.227458,-0.061034,1.307070,-1.849401,1.400679,0.017755,-0.286369,0.585654,0.978695


# Podjela podataka za primjenu modela

In [9]:
X_train=st_X_train
X_test=st_X_test

In [10]:
#Podaci su već podijeljeni u inicijalnom učitavanju - takav je dataset
print(X_train.shape,y_train.shape,X_test.shape,y_test.shape)

(7352, 561) (7352, 1) (2947, 561) (2947, 1)


In [11]:
y_train = y_train.values.ravel()
y_test = y_test.values.ravel()

# LDA

-----***Primjena LDA i logističke regresije***-----

In [12]:
def gridSearch (X_train, y_train, broj_komponenti):
    param_grid = {'penalty':['l2'],
               'C' : [0.01,0.05,0.1], 
               'solver': ['lbfgs','newton-cg','saga'],
                'multi_class': ['multinomial'],
               'max_iter'  : [1000,1500,2000], 
               'class_weight':['balanced']
              }
                  
    log_model = LogisticRegression()
    grid_search = GridSearchCV(log_model,param_grid = param_grid, cv = 5, verbose=2)

    # Treniranje modela sa GridSearchCV
    grid_search.fit(X_train, y_train) 

    rezultati_df = pd.DataFrame(grid_search.cv_results_)

    # Ispis najboljih parametara
    print(f"Најбољи параметри: {grid_search.best_params_}") 
    return grid_search.best_estimator_  

In [13]:
def log_reg_sa_LDA(X_train,y_train,X_test,y_test,broj_komponenti):
        t1 = datetime.datetime.now()
        print('Početak izvršenja: ',t1)
        #Provjerićemo za dobijeni broj komponenti i detaljnije
        print("Dimenzije prije LDA:", X_train.shape)
        LDA = LinearDiscriminantAnalysis(n_components=broj_komponenti) 
        X_train_LDA = LDA.fit_transform(X_train,y_train)
        X_test_LDA = LDA.transform(X_test)
    
        print("Dimenzije nakon LDA:", X_train_LDA.shape)
        print('Матрица добијена примјеном LDA на тестне податке \n', X_test_LDA.round(6))
        print('Коефицијенти \n',LDA.coef_.round(6))
        
        #treniranje modelа 
        t2= datetime.datetime.now()
        print('Početak gridsearch: ',t2)
        najbolji = gridSearch(X_train_LDA, y_train,broj_komponenti)
        t3= datetime.datetime.now()
        print('Kraj gridsearch: ',t3)
        print('Trajanje gridsearch u sekundama ',(t3-t2).total_seconds())

        #tačnost modela na trening podacima
        X_train_LDA_predict = najbolji.predict (X_train_LDA)
        tacnost_training_LDA_data = metrics.accuracy_score(y_train,X_train_LDA_predict).round(6)
        conf_matrica_LDA_training = metrics.confusion_matrix(y_train, X_train_LDA_predict)
        print('Тачност процијењених тренинг података:',tacnost_training_LDA_data)
        
        display(klas_izv(y_train, X_train_LDA_predict, 'тренинг'))
        #display(matrica_predv(conf_matrica_LDA_training))
        #tačnost modela na testnim podacima
        X_test_LDA_predict = najbolji.predict (X_test_LDA)
        tacnost_test_LDA_data = metrics.accuracy_score(y_test,X_test_LDA_predict).round(6)
        conf_matrica_LDA = metrics.confusion_matrix(y_test, X_test_LDA_predict)
        print('Тачност процијењених тестних података:',tacnost_test_LDA_data)
        
        #display(matrica_predv(conf_matrica_LDA))
        
        #klasifikacioni izvjestaj
        
        display(klas_izv(y_test, X_test_LDA_predict, 'тест'))
        t4 = datetime.datetime.now()
        print('Kraj izvršenja: ',t4)
        print('Trajanje izvršenja u sekundama ',(t4-t1).total_seconds())

In [14]:
log_reg_sa_LDA(X_train,y_train,X_test,y_test,broj_komponenti=1)

Početak izvršenja:  2025-04-20 19:22:50.736253
Dimenzije prije LDA: (7352, 561)
Dimenzije nakon LDA: (7352, 1)
Матрица добијена примјеном LDA на тестне податке 
 [[ 12.216972]
 [  9.558249]
 [ 10.545454]
 ...
 [-15.128569]
 [-14.634509]
 [-19.338216]]
Коефицијенти 
 [[ 2.8803000e+00 -2.8820000e-02  7.1643100e-01 ... -2.4313689e+01
  -5.6164746e+01  9.4538600e+00]
 [ 2.8880070e+00 -1.0381780e+00 -5.9604100e-01 ... -7.7742960e+00
  -2.6711554e+01  7.0420650e+00]
 [ 3.3726750e+00 -9.4797400e-01  2.2114300e-01 ... -2.3573603e+01
  -5.1505699e+01  1.9362088e+01]
 [-2.0948510e+00  3.1757100e-01  3.6541600e-01 ...  5.3458889e+01
   2.2915300e+01  7.0999930e+00]
 [-2.0754290e+00  5.9260000e-01  5.9646800e-01 ...  2.8736574e+01
   1.7635029e+01 -6.0984760e+00]
 [-3.1342700e+00  6.1220300e-01 -1.2411590e+00 ... -3.3289416e+01
   6.7238443e+01 -2.7710640e+01]]
Početak gridsearch:  2025-04-20 19:22:53.120913
Fitting 5 folds for each of 27 candidates, totalling 135 fits
[CV] END C=0.01, class_weigh

[CV] END C=0.05, class_weight=balanced, max_iter=1000, multi_class=multinomial, penalty=l2, solver=saga; total time=   0.4s
[CV] END C=0.05, class_weight=balanced, max_iter=1000, multi_class=multinomial, penalty=l2, solver=saga; total time=   0.5s
[CV] END C=0.05, class_weight=balanced, max_iter=1500, multi_class=multinomial, penalty=l2, solver=lbfgs; total time=   0.5s
[CV] END C=0.05, class_weight=balanced, max_iter=1500, multi_class=multinomial, penalty=l2, solver=lbfgs; total time=   0.6s
[CV] END C=0.05, class_weight=balanced, max_iter=1500, multi_class=multinomial, penalty=l2, solver=lbfgs; total time=   0.3s
[CV] END C=0.05, class_weight=balanced, max_iter=1500, multi_class=multinomial, penalty=l2, solver=lbfgs; total time=   0.4s
[CV] END C=0.05, class_weight=balanced, max_iter=1500, multi_class=multinomial, penalty=l2, solver=lbfgs; total time=   0.3s
[CV] END C=0.05, class_weight=balanced, max_iter=1500, multi_class=multinomial, penalty=l2, solver=newton-cg; total time=   0.1

[CV] END C=0.1, class_weight=balanced, max_iter=2000, multi_class=multinomial, penalty=l2, solver=lbfgs; total time=   0.3s
[CV] END C=0.1, class_weight=balanced, max_iter=2000, multi_class=multinomial, penalty=l2, solver=newton-cg; total time=   0.2s
[CV] END C=0.1, class_weight=balanced, max_iter=2000, multi_class=multinomial, penalty=l2, solver=newton-cg; total time=   0.1s
[CV] END C=0.1, class_weight=balanced, max_iter=2000, multi_class=multinomial, penalty=l2, solver=newton-cg; total time=   0.1s
[CV] END C=0.1, class_weight=balanced, max_iter=2000, multi_class=multinomial, penalty=l2, solver=newton-cg; total time=   0.1s
[CV] END C=0.1, class_weight=balanced, max_iter=2000, multi_class=multinomial, penalty=l2, solver=newton-cg; total time=   0.1s
[CV] END C=0.1, class_weight=balanced, max_iter=2000, multi_class=multinomial, penalty=l2, solver=saga; total time=   0.6s
[CV] END C=0.1, class_weight=balanced, max_iter=2000, multi_class=multinomial, penalty=l2, solver=saga; total tim

,Прецизност,Одзив,f1-score,Број узорака у класи
Пјешачење,0.564767,0.533442,0.548658,1226
Пењање уз степенице,0.387234,0.339236,0.361649,1073
Силазак низ степенице,0.635215,0.764706,0.693971,986
Сједење,0.453947,0.482893,0.467973,1286
Стајање,0.494200,0.465066,0.479190,1374
Лежање,0.990043,0.989339,0.989691,1407
Тачност,0.601741,0.601741,0.601741,1
Макро просјек,0.587568,0.595780,0.590189,7352
Пондерисани просјек,0.597120,0.601741,0.598160,7352


Тачност процијењених тестних података: 0.568035


,Прецизност,Одзив,f1-score,Број узорака у класи
Пјешачење,0.494269,0.695565,0.577889,496
Пењање уз степенице,0.631579,0.152866,0.246154,471
Силазак низ степенице,0.398577,0.533333,0.456212,420
Сједење,0.488499,0.908350,0.635328,491
Стајање,0.548387,0.095865,0.163200,532
Лежање,0.945326,0.998138,0.971014,537
Тачност,0.568035,0.568035,0.568035,1
Макро просјек,0.584440,0.564019,0.508300,2947
Пондерисани просјек,0.593576,0.568035,0.513873,2947


Kraj izvršenja:  2025-04-20 19:23:49.436124
Trajanje izvršenja u sekundama  58.699871


In [15]:
log_reg_sa_LDA(X_train,y_train,X_test,y_test,broj_komponenti=2)

Početak izvršenja:  2025-04-20 19:23:49.449146
Dimenzije prije LDA: (7352, 561)
Dimenzije nakon LDA: (7352, 2)
Матрица добијена примјеном LDA на тестне податке 
 [[ 12.216972  -7.913128]
 [  9.558249  -8.322937]
 [ 10.545454  -7.862727]
 ...
 [-15.128569   1.251542]
 [-14.634509   1.085341]
 [-19.338216   3.15337 ]]
Коефицијенти 
 [[ 2.8803000e+00 -2.8820000e-02  7.1643100e-01 ... -2.4313689e+01
  -5.6164746e+01  9.4538600e+00]
 [ 2.8880070e+00 -1.0381780e+00 -5.9604100e-01 ... -7.7742960e+00
  -2.6711554e+01  7.0420650e+00]
 [ 3.3726750e+00 -9.4797400e-01  2.2114300e-01 ... -2.3573603e+01
  -5.1505699e+01  1.9362088e+01]
 [-2.0948510e+00  3.1757100e-01  3.6541600e-01 ...  5.3458889e+01
   2.2915300e+01  7.0999930e+00]
 [-2.0754290e+00  5.9260000e-01  5.9646800e-01 ...  2.8736574e+01
   1.7635029e+01 -6.0984760e+00]
 [-3.1342700e+00  6.1220300e-01 -1.2411590e+00 ... -3.3289416e+01
   6.7238443e+01 -2.7710640e+01]]
Početak gridsearch:  2025-04-20 19:23:51.732651
Fitting 5 folds for each

[CV] END C=0.05, class_weight=balanced, max_iter=1000, multi_class=multinomial, penalty=l2, solver=saga; total time=   1.3s
[CV] END C=0.05, class_weight=balanced, max_iter=1000, multi_class=multinomial, penalty=l2, solver=saga; total time=   1.1s
[CV] END C=0.05, class_weight=balanced, max_iter=1000, multi_class=multinomial, penalty=l2, solver=saga; total time=   0.9s
[CV] END C=0.05, class_weight=balanced, max_iter=1500, multi_class=multinomial, penalty=l2, solver=lbfgs; total time=   0.9s
[CV] END C=0.05, class_weight=balanced, max_iter=1500, multi_class=multinomial, penalty=l2, solver=lbfgs; total time=   0.8s
[CV] END C=0.05, class_weight=balanced, max_iter=1500, multi_class=multinomial, penalty=l2, solver=lbfgs; total time=   0.7s
[CV] END C=0.05, class_weight=balanced, max_iter=1500, multi_class=multinomial, penalty=l2, solver=lbfgs; total time=   0.9s
[CV] END C=0.05, class_weight=balanced, max_iter=1500, multi_class=multinomial, penalty=l2, solver=lbfgs; total time=   0.8s
[CV

C:\Users\Jovana\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


[CV] END C=0.1, class_weight=balanced, max_iter=1000, multi_class=multinomial, penalty=l2, solver=lbfgs; total time=   0.9s
[CV] END C=0.1, class_weight=balanced, max_iter=1000, multi_class=multinomial, penalty=l2, solver=lbfgs; total time=   0.9s
[CV] END C=0.1, class_weight=balanced, max_iter=1000, multi_class=multinomial, penalty=l2, solver=lbfgs; total time=   0.9s
[CV] END C=0.1, class_weight=balanced, max_iter=1000, multi_class=multinomial, penalty=l2, solver=lbfgs; total time=   0.9s
[CV] END C=0.1, class_weight=balanced, max_iter=1000, multi_class=multinomial, penalty=l2, solver=lbfgs; total time=   0.7s
[CV] END C=0.1, class_weight=balanced, max_iter=1000, multi_class=multinomial, penalty=l2, solver=newton-cg; total time=   0.2s
[CV] END C=0.1, class_weight=balanced, max_iter=1000, multi_class=multinomial, penalty=l2, solver=newton-cg; total time=   0.2s
[CV] END C=0.1, class_weight=balanced, max_iter=1000, multi_class=multinomial, penalty=l2, solver=newton-cg; total time=   0

C:\Users\Jovana\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


[CV] END C=0.1, class_weight=balanced, max_iter=1000, multi_class=multinomial, penalty=l2, solver=saga; total time=   1.4s
[CV] END C=0.1, class_weight=balanced, max_iter=1000, multi_class=multinomial, penalty=l2, solver=saga; total time=   1.2s
[CV] END C=0.1, class_weight=balanced, max_iter=1000, multi_class=multinomial, penalty=l2, solver=saga; total time=   1.0s
[CV] END C=0.1, class_weight=balanced, max_iter=1500, multi_class=multinomial, penalty=l2, solver=lbfgs; total time=   0.9s
[CV] END C=0.1, class_weight=balanced, max_iter=1500, multi_class=multinomial, penalty=l2, solver=lbfgs; total time=   0.8s
[CV] END C=0.1, class_weight=balanced, max_iter=1500, multi_class=multinomial, penalty=l2, solver=lbfgs; total time=   0.8s
[CV] END C=0.1, class_weight=balanced, max_iter=1500, multi_class=multinomial, penalty=l2, solver=lbfgs; total time=   0.8s
[CV] END C=0.1, class_weight=balanced, max_iter=1500, multi_class=multinomial, penalty=l2, solver=lbfgs; total time=   0.7s
[CV] END C=

,Прецизност,Одзив,f1-score,Број узорака у класи
Пјешачење,0.816912,0.858891,0.837376,1226
Пењање уз степенице,0.690655,0.599254,0.641717,1073
Силазак низ степенице,0.686385,0.741379,0.712823,986
Сједење,0.584644,0.491446,0.534009,1286
Стајање,0.585814,0.673217,0.626482,1374
Лежање,1.000000,1.000000,1.000000,1407
Тачност,0.733270,0.733270,0.733270,1
Макро просјек,0.727402,0.727365,0.725401,7352
Пондерисани просјек,0.732201,0.733270,0.730760,7352


Тачност процијењених тестних података: 0.612487


,Прецизност,Одзив,f1-score,Број узорака у класи
Пјешачење,0.566138,0.647177,0.603951,496
Пењање уз степенице,0.498127,0.282378,0.360434,471
Силазак низ степенице,0.397790,0.514286,0.448598,420
Сједење,0.548110,0.649695,0.594595,491
Стајање,0.618625,0.524436,0.567650,532
Лежање,1.000000,1.000000,1.000000,537
Тачност,0.612487,0.612487,0.612487,1
Макро просјек,0.604798,0.602995,0.595871,2947
Пондерисани просјек,0.616805,0.612487,0.606946,2947


Kraj izvršenja:  2025-04-20 19:25:31.870856
Trajanje izvršenja u sekundama  102.42171


In [16]:
log_reg_sa_LDA(X_train,y_train,X_test,y_test,broj_komponenti=3)

Početak izvršenja:  2025-04-20 19:25:31.883936
Dimenzije prije LDA: (7352, 561)
Dimenzije nakon LDA: (7352, 3)
Матрица добијена примјеном LDA на тестне податке 
 [[ 12.216972  -7.913128   1.359716]
 [  9.558249  -8.322937  -0.713107]
 [ 10.545454  -7.862727   0.134852]
 ...
 [-15.128569   1.251542   0.077631]
 [-14.634509   1.085341   0.785662]
 [-19.338216   3.15337    1.856824]]
Коефицијенти 
 [[ 2.8803000e+00 -2.8820000e-02  7.1643100e-01 ... -2.4313689e+01
  -5.6164746e+01  9.4538600e+00]
 [ 2.8880070e+00 -1.0381780e+00 -5.9604100e-01 ... -7.7742960e+00
  -2.6711554e+01  7.0420650e+00]
 [ 3.3726750e+00 -9.4797400e-01  2.2114300e-01 ... -2.3573603e+01
  -5.1505699e+01  1.9362088e+01]
 [-2.0948510e+00  3.1757100e-01  3.6541600e-01 ...  5.3458889e+01
   2.2915300e+01  7.0999930e+00]
 [-2.0754290e+00  5.9260000e-01  5.9646800e-01 ...  2.8736574e+01
   1.7635029e+01 -6.0984760e+00]
 [-3.1342700e+00  6.1220300e-01 -1.2411590e+00 ... -3.3289416e+01
   6.7238443e+01 -2.7710640e+01]]
Početa

[CV] END C=0.05, class_weight=balanced, max_iter=1000, multi_class=multinomial, penalty=l2, solver=saga; total time=   1.0s
[CV] END C=0.05, class_weight=balanced, max_iter=1000, multi_class=multinomial, penalty=l2, solver=saga; total time=   1.3s
[CV] END C=0.05, class_weight=balanced, max_iter=1000, multi_class=multinomial, penalty=l2, solver=saga; total time=   0.9s
[CV] END C=0.05, class_weight=balanced, max_iter=1500, multi_class=multinomial, penalty=l2, solver=lbfgs; total time=   0.5s
[CV] END C=0.05, class_weight=balanced, max_iter=1500, multi_class=multinomial, penalty=l2, solver=lbfgs; total time=   0.5s
[CV] END C=0.05, class_weight=balanced, max_iter=1500, multi_class=multinomial, penalty=l2, solver=lbfgs; total time=   0.7s
[CV] END C=0.05, class_weight=balanced, max_iter=1500, multi_class=multinomial, penalty=l2, solver=lbfgs; total time=   0.7s
[CV] END C=0.05, class_weight=balanced, max_iter=1500, multi_class=multinomial, penalty=l2, solver=lbfgs; total time=   0.6s
[CV

C:\Users\Jovana\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


[CV] END C=0.1, class_weight=balanced, max_iter=1000, multi_class=multinomial, penalty=l2, solver=lbfgs; total time=   0.9s
[CV] END C=0.1, class_weight=balanced, max_iter=1000, multi_class=multinomial, penalty=l2, solver=lbfgs; total time=   0.6s
[CV] END C=0.1, class_weight=balanced, max_iter=1000, multi_class=multinomial, penalty=l2, solver=newton-cg; total time=   0.1s
[CV] END C=0.1, class_weight=balanced, max_iter=1000, multi_class=multinomial, penalty=l2, solver=newton-cg; total time=   0.2s
[CV] END C=0.1, class_weight=balanced, max_iter=1000, multi_class=multinomial, penalty=l2, solver=newton-cg; total time=   0.2s
[CV] END C=0.1, class_weight=balanced, max_iter=1000, multi_class=multinomial, penalty=l2, solver=newton-cg; total time=   0.2s
[CV] END C=0.1, class_weight=balanced, max_iter=1000, multi_class=multinomial, penalty=l2, solver=newton-cg; total time=   0.2s
[CV] END C=0.1, class_weight=balanced, max_iter=1000, multi_class=multinomial, penalty=l2, solver=saga; total ti

,Прецизност,Одзив,f1-score,Број узорака у класи
Пјешачење,0.988571,0.987765,0.988168,1226
Пењање уз степенице,0.975948,0.983225,0.979573,1073
Силазак низ степенице,0.994898,0.988844,0.991862,986
Сједење,0.591111,0.517107,0.551638,1286
Стајање,0.595828,0.665211,0.628611,1374
Лежање,1.000000,1.000000,1.000000,1407
Тачност,0.846980,0.846980,0.846980,1
Макро просјек,0.857726,0.857025,0.856642,7352
Пондерисани просјек,0.846843,0.846980,0.846119,7352


Тачност процијењених тестних података: 0.788599


,Прецизност,Одзив,f1-score,Број узорака у класи
Пјешачење,0.953596,0.828629,0.886731,496
Пењање уз степенице,0.771993,0.912951,0.836576,471
Силазак низ степенице,0.946154,0.878571,0.911111,420
Сједење,0.558912,0.376782,0.450122,491
Стајање,0.559201,0.736842,0.635848,532
Лежање,1.000000,1.000000,1.000000,537
Тачност,0.788599,0.788599,0.788599,1
Макро просјек,0.798309,0.788963,0.786731,2947
Пондерисани просјек,0.795011,0.788599,0.784796,2947


Kraj izvršenja:  2025-04-20 19:27:00.391823
Trajanje izvršenja u sekundama  88.507887


In [17]:
log_reg_sa_LDA(X_train,y_train,X_test,y_test,broj_komponenti=4)

Početak izvršenja:  2025-04-20 19:27:00.398649
Dimenzije prije LDA: (7352, 561)
Dimenzije nakon LDA: (7352, 4)
Матрица добијена примјеном LDA на тестне податке 
 [[ 12.216972  -7.913128   1.359716   0.905909]
 [  9.558249  -8.322937  -0.713107  -0.668969]
 [ 10.545454  -7.862727   0.134852   0.020767]
 ...
 [-15.128569   1.251542   0.077631  -4.233406]
 [-14.634509   1.085341   0.785662  -3.572347]
 [-19.338216   3.15337    1.856824  -4.529084]]
Коефицијенти 
 [[ 2.8803000e+00 -2.8820000e-02  7.1643100e-01 ... -2.4313689e+01
  -5.6164746e+01  9.4538600e+00]
 [ 2.8880070e+00 -1.0381780e+00 -5.9604100e-01 ... -7.7742960e+00
  -2.6711554e+01  7.0420650e+00]
 [ 3.3726750e+00 -9.4797400e-01  2.2114300e-01 ... -2.3573603e+01
  -5.1505699e+01  1.9362088e+01]
 [-2.0948510e+00  3.1757100e-01  3.6541600e-01 ...  5.3458889e+01
   2.2915300e+01  7.0999930e+00]
 [-2.0754290e+00  5.9260000e-01  5.9646800e-01 ...  2.8736574e+01
   1.7635029e+01 -6.0984760e+00]
 [-3.1342700e+00  6.1220300e-01 -1.24115

[CV] END C=0.05, class_weight=balanced, max_iter=1000, multi_class=multinomial, penalty=l2, solver=saga; total time=   0.9s
[CV] END C=0.05, class_weight=balanced, max_iter=1000, multi_class=multinomial, penalty=l2, solver=saga; total time=   0.6s
[CV] END C=0.05, class_weight=balanced, max_iter=1000, multi_class=multinomial, penalty=l2, solver=saga; total time=   0.9s
[CV] END C=0.05, class_weight=balanced, max_iter=1000, multi_class=multinomial, penalty=l2, solver=saga; total time=   0.6s
[CV] END C=0.05, class_weight=balanced, max_iter=1500, multi_class=multinomial, penalty=l2, solver=lbfgs; total time=   0.4s
[CV] END C=0.05, class_weight=balanced, max_iter=1500, multi_class=multinomial, penalty=l2, solver=lbfgs; total time=   0.5s
[CV] END C=0.05, class_weight=balanced, max_iter=1500, multi_class=multinomial, penalty=l2, solver=lbfgs; total time=   0.2s
[CV] END C=0.05, class_weight=balanced, max_iter=1500, multi_class=multinomial, penalty=l2, solver=lbfgs; total time=   0.4s
[CV]

[CV] END C=0.1, class_weight=balanced, max_iter=2000, multi_class=multinomial, penalty=l2, solver=lbfgs; total time=   0.2s
[CV] END C=0.1, class_weight=balanced, max_iter=2000, multi_class=multinomial, penalty=l2, solver=lbfgs; total time=   0.3s
[CV] END C=0.1, class_weight=balanced, max_iter=2000, multi_class=multinomial, penalty=l2, solver=lbfgs; total time=   0.5s
[CV] END C=0.1, class_weight=balanced, max_iter=2000, multi_class=multinomial, penalty=l2, solver=newton-cg; total time=   0.1s
[CV] END C=0.1, class_weight=balanced, max_iter=2000, multi_class=multinomial, penalty=l2, solver=newton-cg; total time=   0.1s
[CV] END C=0.1, class_weight=balanced, max_iter=2000, multi_class=multinomial, penalty=l2, solver=newton-cg; total time=   0.1s
[CV] END C=0.1, class_weight=balanced, max_iter=2000, multi_class=multinomial, penalty=l2, solver=newton-cg; total time=   0.1s
[CV] END C=0.1, class_weight=balanced, max_iter=2000, multi_class=multinomial, penalty=l2, solver=newton-cg; total t

,Прецизност,Одзив,f1-score,Број узорака у класи
Пјешачење,1.000000,1.000000,1.000000,1226
Пењање уз степенице,1.000000,1.000000,1.000000,1073
Силазак низ степенице,1.000000,1.000000,1.000000,986
Сједење,0.659738,0.666407,0.663056,1286
Стајање,0.684791,0.678311,0.681536,1374
Лежање,1.000000,1.000000,1.000000,1407
Тачност,0.881529,0.881529,0.881529,1
Макро просјек,0.890755,0.890786,0.890765,7352
Пондерисани просјек,0.881573,0.881529,0.881545,7352


Тачност процијењених тестних података: 0.833051


,Прецизност,Одзив,f1-score,Број узорака у класи
Пјешачење,0.988814,0.891129,0.937434,496
Пењање уз степенице,0.864564,0.989384,0.922772,471
Силазак низ степенице,0.982005,0.909524,0.944376,420
Сједење,0.551893,0.920570,0.690076,491
Стајање,0.814815,0.330827,0.470588,532
Лежање,1.000000,1.000000,1.000000,537
Тачност,0.833051,0.833051,0.833051,1
Макро просјек,0.867015,0.840239,0.827541,2947
Пондерисани просјек,0.865818,0.833051,0.821992,2947


Kraj izvršenja:  2025-04-20 19:28:02.500578
Trajanje izvršenja u sekundama  62.101929


In [18]:
log_reg_sa_LDA(X_train,y_train,X_test,y_test,broj_komponenti=5)

Početak izvršenja:  2025-04-20 19:28:02.508556
Dimenzije prije LDA: (7352, 561)
Dimenzije nakon LDA: (7352, 5)
Матрица добијена примјеном LDA на тестне податке 
 [[ 12.216972  -7.913128   1.359716   0.905909  -4.640849]
 [  9.558249  -8.322937  -0.713107  -0.668969  -3.237235]
 [ 10.545454  -7.862727   0.134852   0.020767  -1.902887]
 ...
 [-15.128569   1.251542   0.077631  -4.233406   0.566722]
 [-14.634509   1.085341   0.785662  -3.572347   0.681124]
 [-19.338216   3.15337    1.856824  -4.529084   0.253822]]
Коефицијенти 
 [[ 2.8803000e+00 -2.8820000e-02  7.1643100e-01 ... -2.4313689e+01
  -5.6164746e+01  9.4538600e+00]
 [ 2.8880070e+00 -1.0381780e+00 -5.9604100e-01 ... -7.7742960e+00
  -2.6711554e+01  7.0420650e+00]
 [ 3.3726750e+00 -9.4797400e-01  2.2114300e-01 ... -2.3573603e+01
  -5.1505699e+01  1.9362088e+01]
 [-2.0948510e+00  3.1757100e-01  3.6541600e-01 ...  5.3458889e+01
   2.2915300e+01  7.0999930e+00]
 [-2.0754290e+00  5.9260000e-01  5.9646800e-01 ...  2.8736574e+01
   1.76

[CV] END C=0.05, class_weight=balanced, max_iter=1000, multi_class=multinomial, penalty=l2, solver=saga; total time=   0.5s
[CV] END C=0.05, class_weight=balanced, max_iter=1000, multi_class=multinomial, penalty=l2, solver=saga; total time=   0.0s
[CV] END C=0.05, class_weight=balanced, max_iter=1000, multi_class=multinomial, penalty=l2, solver=saga; total time=   0.6s
[CV] END C=0.05, class_weight=balanced, max_iter=1000, multi_class=multinomial, penalty=l2, solver=saga; total time=   0.6s
[CV] END C=0.05, class_weight=balanced, max_iter=1500, multi_class=multinomial, penalty=l2, solver=lbfgs; total time=   0.1s
[CV] END C=0.05, class_weight=balanced, max_iter=1500, multi_class=multinomial, penalty=l2, solver=lbfgs; total time=   0.3s
[CV] END C=0.05, class_weight=balanced, max_iter=1500, multi_class=multinomial, penalty=l2, solver=lbfgs; total time=   0.1s
[CV] END C=0.05, class_weight=balanced, max_iter=1500, multi_class=multinomial, penalty=l2, solver=lbfgs; total time=   0.2s
[CV]

[CV] END C=0.1, class_weight=balanced, max_iter=2000, multi_class=multinomial, penalty=l2, solver=lbfgs; total time=   0.3s
[CV] END C=0.1, class_weight=balanced, max_iter=2000, multi_class=multinomial, penalty=l2, solver=lbfgs; total time=   0.4s
[CV] END C=0.1, class_weight=balanced, max_iter=2000, multi_class=multinomial, penalty=l2, solver=lbfgs; total time=   0.3s
[CV] END C=0.1, class_weight=balanced, max_iter=2000, multi_class=multinomial, penalty=l2, solver=newton-cg; total time=   0.1s
[CV] END C=0.1, class_weight=balanced, max_iter=2000, multi_class=multinomial, penalty=l2, solver=newton-cg; total time=   0.1s
[CV] END C=0.1, class_weight=balanced, max_iter=2000, multi_class=multinomial, penalty=l2, solver=newton-cg; total time=   0.1s
[CV] END C=0.1, class_weight=balanced, max_iter=2000, multi_class=multinomial, penalty=l2, solver=newton-cg; total time=   0.1s
[CV] END C=0.1, class_weight=balanced, max_iter=2000, multi_class=multinomial, penalty=l2, solver=newton-cg; total t

,Прецизност,Одзив,f1-score,Број узорака у класи
Пјешачење,1.000000,1.000000,1.000000,1226
Пењање уз степенице,1.000000,1.000000,1.000000,1073
Силазак низ степенице,1.000000,1.000000,1.000000,986
Сједење,0.957725,0.968896,0.963278,1286
Стајање,0.970567,0.959971,0.965240,1374
Лежање,1.000000,1.000000,1.000000,1407
Тачност,0.987078,0.987078,0.987078,1
Макро просјек,0.988049,0.988144,0.988086,7352
Пондерисани просјек,0.987105,0.987078,0.987080,7352


Тачност процијењених тестних података: 0.931456


,Прецизност,Одзив,f1-score,Број узорака у класи
Пјешачење,0.993258,0.891129,0.939426,496
Пењање уз степенице,0.857934,0.987261,0.918065,471
Силазак низ степенице,0.979221,0.897619,0.936646,420
Сједење,0.971154,0.822811,0.890849,491
Стајање,0.836013,0.977444,0.901213,532
Лежање,1.000000,1.000000,1.000000,537
Тачност,0.931456,0.931456,0.931456,1
Макро просјек,0.939597,0.929377,0.931033,2947
Пондерисани просјек,0.938789,0.931456,0.931662,2947


Kraj izvršenja:  2025-04-20 19:28:48.653576
Trajanje izvršenja u sekundama  46.14502
